<a href="https://colab.research.google.com/github/carl03111/unsloth/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B", # This loads the base Qwen3-14B model
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False, # Ensure this is False
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [3]:
messages = [
    {"role" : "user", "content" : "说明合成孔径雷达（SAR）中方位向分辨率的提升原理及实现条件,高分辨率地表成像需求"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = True,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048,
    temperature = 0.6, top_p = 0.95, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
嗯，用户问的是关于合成孔径雷达（SAR）中方位向分辨率的提升原理和实现条件，以及高分辨率地表成像的需求。首先，我需要回忆一下SAR的基本原理。SAR通过移动平台（比如卫星或飞机）上的雷达发射信号，并接收地表反射回来的信号，利用平台的运动合成一个更大的天线，从而提高分辨率。

方位向分辨率，也就是沿着雷达运动方向的分辨率，对吧？我记得SAR的分辨率主要分为距离向和方位向。距离向分辨率由雷达波长和天线孔径决定，而方位向分辨率则与合成孔径的长度有关。所以提升方位向分辨率可能需要增加合成孔径的长度，也就是让雷达平台运动更长的距离，或者通过其他技术手段。

接下来，用户提到的实现条件可能包括平台的速度、合成孔径的时间、信号处理技术等。比如，平台速度越快，合成孔径的时间越长，可能可以合成更长的天线，从而提高分辨率。不过这里可能有矛盾，因为合成孔径的时间和平台速度的关系需要仔细分析。另外，信号处理方面，可能需要使用脉冲压缩、多普勒处理等技术。

高分辨率地表成像需求方面，用户可能想知道为什么需要高分辨率。比如，用于城市监测、灾害评估、军事侦察等场景，这些都需要更清晰的图像来识别小目标或细节。不过，用户可能还想知道高分辨率带来的挑战，比如数据量大、处理复杂度高，或者对雷达系统的要求更高。

可能用户是学生或者工程师，正在研究SAR技术，或者需要应用SAR数据。他们可能希望了解如何在实际中提升方位向分辨率，以及高分辨率对应用的影响。需要确保解释清楚原理、条件以及应用场景，避免过于技术化的术语，但也要准确。

需要确认方位向分辨率提升的关键因素，比如合成孔径长度、脉冲重复频率、多普勒带宽等。可能还要提到像距压缩技术，比如使用Chirp信号进行脉冲压缩，或者使用多普勒处理来提高分辨率。

另外，用户可能混淆了距离向和方位向分辨率的区别，需要明确区分。方位向分辨率主要受合成孔径长度的影响，而距离向则受发射信号带宽影响。提升方位向分辨率需要更长的合成孔径，这可能通过增加平台飞行时间或使用更宽的多普勒带宽来实现。

最后，高分辨率成像的需求可能涉及不同的应用场景，需要举例说明，比如城市监测需要分辨建筑物，农业监测需要分辨作物类型等。同时，可能提到SAR的缺点，比如对地表覆盖的限制，或者需要配合其他遥感数据使用。
</think>

合成孔径雷达（SAR）是一种主动遥感技术，通过